## för implementering i streamlit
```py
import streamlit as st

st.markdown("### 🔍 AI-analys: Vanliga krav & meriter")

st.info(data["sammanfattning"][0])

st.subheader("📌 Topp 5 krav")
st.bar_chart(pd.DataFrame({"Antal": data["antal krav"]}, index=data["krav"]))

st.subheader("📌 Topp 5 meriter")
st.bar_chart(pd.DataFrame({"Antal": data["antal meriterande"]}, index=data["meriterande"]))

In [49]:
from google import genai
import os
from dotenv import load_dotenv
import duckdb
import plotly_express as px
import pandas as pd
import json
from google import generativeai as oldGenai
from google.generativeai.types import GenerationConfig

# läs in data och importera nödvändiga paket
with duckdb.connect("../../job_ads_data_warehouse.duckdb") as con: 
    df = con.execute("SELECT * FROM mart.mart_full_job_ads").df()
df_transport = df[df["occupation_field"] == "Transport, distribution, lager"]
df_hotel = df[df["occupation_field"] == "Hotell, restaurang, storhushåll"]
df_install = df[df["occupation_field"] == "Installation, drift, underhåll"]

# Prompt 1: Sammanställning för ett occupation_field
### vald/filtrerad df, prompt, df för skills, sammanställd text och piechart

In [144]:
# skapa objekt för gemini med api key
load_dotenv()
client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

# filtrera df
current_df = df_transport[["occupation_field", "headline", "description"]].iloc[:10]

# lägg in info till prompt för generell sammanställning för översikt
promt = f"""Du är en rekryterare inom {current_df["occupation_field"]}.
Plocka ut max 5 av dom vanligast nämnda kraven och max 5 av dom vanligast nämnda meriterade/eftertraktade färdigheterna,
och ett sammanställt antal av dessa som efterfrågas i dessa jobbannonser {current_df["description"]}, detta ska enkelt kunna användas till en plot i en dashboard
skriv även en sammanfattning av krav och meriter som du skriver ut för jobb fältet för att tydligt beskriva vad som sökt och är mest eftertraktat just nu enligt annonserna:

Output ska vara i detta formatet enbart: 

{{
    "sammanfattning": "sammanfattnings text",
    "krav": [erfarenhet1, erfarenhet2, ...],
    "antal krav": [summa erfarenhet1, erfarenhet2..],
    "meriterande": [meriterande1,meriterande2, ... ],
    "antal meriterande": [summa meriterande1,meriterande2..]
    
}}
"""
# skicka in prompt till gemini
model = oldGenai.GenerativeModel(
        model_name="gemini-2.0-flash",
        generation_config=GenerationConfig(
            temperature=0.0,         
            top_p=1.0,  # Säkerställer att sampling inte påverkas   #verkar inte göra skillnad
            top_k=1     # Tar bort variation)                       #verkar inte göra skillnad    
        )
    )
response = model.generate_content(promt)

# städa response text från gemini
cleaned = response.text.replace("\n", "").strip("```json")
data = json.loads(cleaned)

# strukturera upp datan i dfs för plots och en text-string för sammanfattning
krav_df = pd.DataFrame({
    "Krav": data["krav"],
    "Antal": data["antal krav"]
})
merit_df = pd.DataFrame({
    "Krav": data["meriterande"],
    "Antal": data["antal meriterande"]
})
summary = data["sammanfattning"]

# skriv ut sammanfattning och plots
print(summary)
fig1 = px.pie(krav_df, names="Krav", values="Antal", title="Sammanställning av krav på färdigheter för annonser")
fig2 = px.pie(merit_df, names="Krav", values="Antal", title="Sammanställning av meriterande färdigheter för annonser")
fig1.show(), fig2.show()

Inom transport, distribution och lager efterfrågas främst erfarna chaufförer med B-körkort. Truckkort och erfarenhet av lagerarbete är starkt meriterande. Förmåga att arbeta självständigt och vara serviceinriktad är också viktiga egenskaper.


(None, None)

In [ ]:
print(summary)
fig1 = px.bar(krav_df, x="Krav", y="Antal", title="Sammanställning av krav på färdigheter för annonser", color="Antal", color_continuous_scale="Teal")
fig2 = px.bar(merit_df, x="Krav", y="Antal", title="Sammanställning av meriterande färdigheter för annonser", color="Krav", color_continuous_scale=px.colors.sequential.thermal)
fig1.layout
fig1.show(), fig2.show()

Inom transport, distribution och lager efterfrågas främst erfarna chaufförer med B-körkort. Truckkort och erfarenhet av lagerarbete är starkt meriterande. Förmåga att arbeta självständigt och vara serviceinriktad är också viktiga egenskaper.


In [135]:
print(summary)
fig1 = px.pie(krav_df, names="Krav", values="Antal", title="Sammanställning av krav på färdigheter för annonser", color="Antal")
fig2 = px.pie(merit_df, names="Krav", values="Antal", title="Sammanställning av meriterande färdigheter för annonser", color="Antal", color_discrete_sequence=px.colors.sequential.Plasma)
fig1.show(), fig2.show()

Inom transport, distribution och lager efterfrågas främst erfarna chaufförer med B-körkort. Truckkort och erfarenhet av lagerarbete är starkt meriterande. Förmåga att arbeta självständigt och vara serviceinriktad är också viktiga egenskaper.


(None, None)